# Studies

## Dataset

In [1]:
from tqdm.notebook import tqdm
from dataset import get_dataset

In [4]:
def search_seed():
    try:
        with open('primes.txt') as f:
            primes = [int(x) for x in ' '.join(f.readlines()).split()]
    except FileNotFoundError:
        print('WARNING: File primes.txt not found, you can find a list of primes as https://t5k.org/lists/small/10000.txt')
        print('INFO: seeds will fallback to the list [1, 10_000]')
        primes = [i+1 for i in range(10_000)]

    curr_diff, selected_prime = 9999, None
    for p in tqdm(primes):
        new_diff = get_dataset(seed=p, get_deviation=True)
        if new_diff < curr_diff:
            curr_diff, selected_prime = new_diff, p

    return selected_prime, curr_diff

In [ ]:
search_seed()

## Quantum Kernel

In [ ]:
# Kernel
from qkernel import get_quantum_kernel

# QSVM
from qiskit_machine_learning.algorithms import PegasosQSVC

# Param Optimization
import time
from sklearn.metrics import accuracy_score

# Dataset
from sklearn.decomposition import PCA
from dataset import get_dataset

In [3]:
def run_qsvm(shots=5, entanglement='circular', num_qubits=2, big_c=256, reps=1, f_map='z_map'):
    print(f'LOG PARAM: num_qubits {num_qubits} - big_c {big_c} - reps {reps} - f_map {f_map} - entanglement {entanglement} - shots {shots}')

    train_features, test_features, train_labels, test_labels = get_dataset()
    if num_qubits != train_features.shape[1]:
        pca = PCA(n_components=num_qubits)
        train_features = pca.fit_transform(train_features)
        test_features = pca.transform(test_features)

    start = time.perf_counter()
    qkernel = get_quantum_kernel(shots=shots, f_map_name=f_map, num_qubits=num_qubits, reps=reps, entanglement=entanglement)

    model = PegasosQSVC(quantum_kernel=qkernel, C=big_c)
    model.fit(train_features, train_labels)
    duration = round(time.perf_counter() - start, 3)
    accuracy = accuracy_score(test_labels, model.predict(test_features))

    print(f'LOG RESULTS: Accuracy {accuracy} - Duration {duration}')

In [ ]:
for num_qubits_ in [2, 4]:
    for reps_ in [1, 2, 3]:
        for big_c_ in [256, 1024, 4096]:
            for f_map_ in ['z_map', 'zz_map']:
                for entanglement_ in ['linear', 'circular', 'full']:
                    run_qsvm(num_qubits=num_qubits_, reps=reps_, big_c=big_c_, f_map=f_map_, entanglement=entanglement_)

runs = [
    {'num_qubits': 8, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'linear', 'big_c': 1024},
    {'num_qubits': 8, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'linear', 'big_c': 4096},
    {'num_qubits': 8, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'full', 'big_c': 4096},
    {'num_qubits': 16, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'linear', 'big_c': 4096},
    {'num_qubits': 16, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'full', 'big_c': 4096},
    {'num_qubits': 30, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'linear', 'big_c': 4096},
    {'num_qubits': 30, 'reps': 3, 'f_map': 'z_map', 'entanglement': 'linear', 'big_c': 4096},
    {'num_qubits': 30, 'reps': 1, 'f_map': 'zz_map', 'entanglement': 'linear', 'big_c': 4096},
    {'num_qubits': 30, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'linear', 'big_c': 256},
    {'num_qubits': 30, 'reps': 2, 'f_map': 'z_map', 'entanglement': 'linear', 'big_c': 1024},
    {'num_qubits': 30, 'reps': 2, 'f_map': 'zz_map', 'entanglement': 'linear', 'big_c': 4096},
]
for r in runs:
    run_qsvm(**r)

## Quantum Support Vector Machine

In [1]:
from qsvm import QASVM
from dataset import get_dataset
from sklearn.metrics import f1_score

In [ ]:
train_features, test_features, train_labels, test_labels = get_dataset()
for big_c in range(1, 13):
    svm = QASVM(big_c=2**big_c)
    svm.fit(train_features, train_labels)
    predictions = svm.predict(test_features)
    print(f'C: {2**big_c} - F1: {round(f1_score(test_labels, predictions), 3)}')